labelimg 로 이미지 라벨링 하고,   
xml 로 저장한 좌표 정보를   
txt 로 좌표를 변경해서 생성해주는 프로그램

In [1]:
'''
함수부, 한번만 실행하세요
'''

from PIL import Image
import os
import progressbar

def yolo_xml2txt_coordinate(size, bbox):
    '''
    size = (w, h)
    bbox = (xmin, xmax, ymin, ymax)
    x1, y1, x2, y2
    center_x, center_y, w, h = yolo_xml2txt_coordinate(size, bbox)
    '''
    dw = 1. / size[0]  # 이미지 가로 길이
    dh = 1. / size[1]  # 이미지 세로 길이
    
    # 중심 좌표 상대값 구하기
    center_x = (bbox[0] + bbox[2]) / 2.0
    center_y = (bbox[1] + bbox[3]) / 2.0
    center_x = center_x * dw
    center_y = center_y * dh
    
    # 바운딩 박스 가로, 세로 상대값 구하기
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    w = w * dw
    h = h * dh
    
    return center_x, center_y, w, h


def txt_coordinate_create(dest_path, img_name, bbox):
    '''
    txt 파일 생성
    '''
    # img_name.jpg 에서 img_name 만 분리
    file_name, ext = os.path.splitext(img_name)
    
    # /path/img_name.xml 생성
    path_name = os.path.join(dest_path, file_name +'.txt')
    
    # 쓰기 파일 열기
    f = open(path_name, 'wt', encoding='utf-8')
    
    for box in bbox:
        lines = ['00 ' + box[0] +' '+ box[1] +' '+ box[2] +' '+ box[3] +'\n']
        f.writelines(lines)
    
    f.close()
    

def f_get_filename_img(dest_path='./', img_mode=True):
    '''
    설명: 특정 디렉토리에서 이미지 파일 여부에 따라 파일명 반환
    입력: 경로명, img_mode
    출력: 파일명 리스트
    옵션: img_mode = True -> 이미지 파일명 반환
         img_mode = False -> 이미지 파일이 아닌 파일의 파일명 반환
    예시: img_list = f_get_filename_img('./data', img_mode=True)
         print(img_list)
    '''
    import imghdr
    import os
    
    files_list = os.listdir(dest_path)
    
    ret_list = []  # 반환 할 리스트
    
    for file_name in files_list:
        path_name = os.path.join(dest_path, file_name)
        
        if imghdr.what(path_name) == None:  # 이미지 파일이 아니면
            if img_mode == False:
                ret_list.append(file_name)
        else:                               # 이미지 파일이면
            if img_mode == True:
                ret_list.append(file_name)
                
    return ret_list



def f_read_anntation_xml(xml_file: str):
    '''
    '''
    import xml.etree.ElementTree as ET
    
    # print('read_anntation:', str)
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []
    
    # 파일 이름
    file_name = root.find('filename').text
    
    # 이미지 사이즈 값 추출
    size_tag = root.find('size')
    w = int(size_tag.find('width').text)
    h = int(size_tag.find('height').text)
    c = int(size_tag.find('depth').text)
    size_list = [w, h, c]
    
    # object 태그 집합
    for obj in root.iter('object'):
        
        # object >> name 태그
        object_label = obj.find("name").text
        
        # object >> 바운드 박스 값 추출
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name, size_list

In [6]:
'''
아래 예시대로 폴더 경로 설정 꼭 하세요. '/' 주의
file_path = './alyac_aug/aug_20장_jpg_xml_장방_103~210/'

폴더 안에 알약 이미지 파일, 이미지 파일과 같은 이름의 xml 파일 있어야 합니다.

'''
file_path = './alyac_aug/aug_20장_jpg_xml_장방_103~210/'
img_list = f_get_filename_img(file_path, img_mode=True)

bar = progressbar.ProgressBar(maxval=len(img_list)).start()

for i, img_name in enumerate(img_list):
    # xml 파일 존재 유무 확인
    xml_name, ext = os.path.splitext(img_name)
    xml_name = xml_name+'.xml'
    if os.path.isfile(os.path.join(file_path, xml_name)) == False:
        print('xml 파일이 없습니다: ', txt_name)
        continue
        
    # xml 파일 파싱 >> 이미지 파일명, 이미지 크기, 바운딩 박스 정보 추출
    xml_path_name = os.path.join(file_path, xml_name)
    bounding_box_list, img_name, size_list = f_read_anntation_xml(xml_path_name)
    
    # 바운딩 박스 좌표 변환
    txt_bbox = []
    for bbox in bounding_box_list:
        bbox = bbox[1:]
        tmp_x, tmp_y, tmp_w, tmp_h = yolo_xml2txt_coordinate(size_list, bbox)
        
        tmp_x = str(format(round(tmp_x, 6), '.6f'))
        tmp_y = str(format(round(tmp_y, 6), '.6f'))
        tmp_w = str(format(round(tmp_w, 6), '.6f'))
        tmp_h = str(format(round(tmp_h, 6), '.6f'))
        
        tmp_bbox = [tmp_x, tmp_y, tmp_w, tmp_h]
        txt_bbox.append(tmp_bbox)
        
    # txt 파일 생성
    txt_coordinate_create(file_path, img_name, txt_bbox)
    
    bar.update(i)

bar.finish()

100% |########################################################################|
